In [90]:
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
#from .utils import PARAMETER_NAMES_ALL_PRECESSINGBNS_BILBY
from river.models.utils import *
from river.data.utils import *

import river.data
from river.data.datagenerator import DataGeneratorBilbyFD
from river.data.dataset import DatasetSVDStrainFDFromSVDWFonGPU #DatasetStrainFD, #DatasetStrainFDFromPreCalSVDWF
#import river.data.utils as datautils
from river.data.utils import *

from river.models import embedding
from river.models.utils import *
#from river.models.embedding.pca import project_strain_data_FDAPhi
from river.models.embedding.conv import EmbeddingConv1D, EmbeddingConv2D
from river.models.embedding.mlp import EmbeddingMLP1D, ResidualMLPBlock1D

from glasflow.nflows.nn.nets import ResidualNet, ConvResidualNet
from glasflow.nflows.nn.nets.resnet import ResidualBlock

In [92]:
ResidualBlock??

In [2]:
import numpy as np
import bilby 
import pycbc 
import sys
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import nn
import torch.nn.functional as F


import pickle
#from sklearn.utils.extmath import randomized_svd
import sklearn
from sklearn.decomposition import IncrementalPCA, randomized_svd, KernelPCA
import sklearn.decomposition 

import glob
import time

In [3]:
validfolder = '/home/qian.hu/mlpe/training_data/bns_50Hz1024Hz32s_lowspin_lb/valid'
trainfolder = '/home/qian.hu/mlpe/training_data/bns_50Hz1024Hz32s_lowspin_lb/train'

valid_filenames = glob.glob(f"{validfolder}/batch*/*.h5")
train_filenames = glob.glob(f"{trainfolder}/batch*/*.h5")



In [4]:
source_type = 'BNS'
detector_names = ['H1', 'L1', 'V1'] 
duration = 32
f_low = 50
f_high = 1024

f_ref = 20
sampling_frequency = 2048
waveform_approximant = 'IMRPhenomPv2_NRTidal'
parameter_names = PARAMETER_NAMES_ALL_PRECESSINGBNS_BILBY
PSD_type = 'bilby_default' #'zero_noise' bilby_default
use_sealgw_detector = True

Vhfile="/home/qian.hu/mlpe/river/test/outputs/Vh_50Hz1024Hz32s.pickle"

data_generator = DataGeneratorBilbyFD(source_type,
            detector_names, 
            duration, 
            f_low, 
            f_ref, 
            sampling_frequency, 
            waveform_approximant, 
            parameter_names,
            PSD_type=PSD_type,
            use_sealgw_detector=use_sealgw_detector,
            f_high=f_high)


21:48 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_neutron_star
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


Using bilby_default PSDs to generate data.


In [5]:
device='cuda'

dataset = DatasetSVDStrainFDFromSVDWFonGPU(valid_filenames, PARAMETER_NAMES_CONTEXT_PRECESSINGBNS_BILBY, data_generator,
                                     Nbasis=512, Vhfile=Vhfile, device=device)

batch_size = 4
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [6]:
theta, x = dataset[1]

for theta_batch, x_batch in dataloader:
    pass

In [7]:
x_batch.shape

torch.Size([4, 6, 512])

In [8]:
#embd = EmbeddingConv1D(ndet=3, nout=256, num_blocks=2, use_psd = False).to(device)


In [9]:
import glasflow

In [75]:
class NewResidualMLPBlock1D(nn.Module):
    def __init__(self, in_features, out_features, num_blocks):
        super().__init__()
        self.blocks = nn.ModuleList([nn.Sequential(
            nn.Linear(in_features if i == 0 else out_features, out_features),
            nn.BatchNorm1d(out_features),
            nn.ReLU(),
            nn.Linear(out_features, out_features),
            nn.BatchNorm1d(out_features)
        ) for i in range(num_blocks)])
        self.reshape = nn.Linear(in_features, out_features)  # Linear reshape for residual

    def forward(self, x):
        for i, block in enumerate(self.blocks):
            residual = x
            if i == 0:
                out = block(x)
            else:
                out = block(out)  # Use the output from the previous block as input
            residual = self.reshape(residual)  # Apply reshape to residual
            out += residual
            out = F.relu(out)
        return out

    

class EmbeddingResConv1DMLP(nn.Module):
    def __init__(self, nbasis, nout,conv_params, mlp_params, **kwargs):
        super().__init__()
        self.nout = nout
        #self.nchannel = 2*ndet
        self.nbasis = nbasis
        
        self.conv_params = conv_params        
        self.mlp_params = mlp_params
        
        self.conv = self.make_conv_layer()
        
        with torch.no_grad():
            testx = torch.randn((1, self.conv_params['in_channel'][0], self.nbasis))
            self.mlp_params['in_features'][0] = self.conv(testx).shape[-1] * self.conv_params['out_channel'][-1]
            print(f"Initialized MLP in channel: {self.mlp_params['in_features'][0]}")

        
        self.mlp = self.make_mlp_layer()

    def make_conv_layer(self):
        layers = []
        for i in range(len(self.conv_params['in_channel'])):
            layers.append(ResidualConvBlock1D(in_channels=self.conv_params['in_channel'][i], 
                                              out_channels=self.conv_params['out_channel'][i], 
                                              kernel_size=self.conv_params['kernel_size'][i], 
                                              stride=self.conv_params['stride'][i], 
                                              padding=self.conv_params['padding'][i], 
                                              dilation=self.conv_params['dilation'][i], 
                                              dropout=self.conv_params['dropout'][i]))
        
        return nn.Sequential(*layers)

    def make_mlp_layer(self):
        layers = []
        for i in range(len(self.mlp_params['in_features'])):
            layers.append(nn.Linear(self.mlp_params['in_features'][i], self.mlp_params['out_features'][i]))
            if i!= len(self.mlp_params['in_features'])-1:
                layers.append(nn.ReLU())
            
            
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv(x)
        x = x.flatten(start_dim=1)
        out = self.mlp(x)

        return out

In [89]:
ResidualBlock??

Object `ResidualBlock` not found.


In [123]:
conv_params = {
            'in_channel':  [6, 128, 128, 128, 64, 64, 64, 32, 32, 32, 16, 16, 16, 8, 8, 8, 4, 4, 4],
            'out_channel': [128, 128, 128, 64, 64, 64, 32, 32, 32, 16, 16, 16, 8, 8, 8, 4, 4, 4, 1],
            'kernel_size': [32, 16, 16, 16, 8, 8, 8, 8, 8, 8, 8, 4, 4, 4, 4, 2, 2, 2, 1],
            'stride':      [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
            'padding':     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            'dilation':    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
            'dropout':     [0.1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        }
mlp_params =  {
            'in_features': [0,],
            'out_features': [256,],
        }

In [122]:
conv_params = {
            'in_channel': [6, ],
            'out_channel': [1,],
            'kernel_size': [16,],
            'stride': [1, ],
            'padding': [0, ],
            'dilation': [1, ],
            'dropout': [0.1,]
        }
mlp_params =  {
            'in_features': [0,],
            'out_features': [256,],
        }

In [124]:
embd = EmbeddingResConv1DMLP( 512, 256, conv_params, mlp_params).to(device)

Initialized MLP in channel: 372


In [125]:
y = embd(x_batch)

In [126]:
y.shape

torch.Size([4, 256])

In [127]:
count_parameters(embd)

1739436

In [130]:
import glasflow

In [131]:
glasflow.CouplingNSF?

In [76]:
y

tensor([[-0.2815,  0.0368, -0.5166,  ..., -0.4752, -0.2145,  0.1430],
        [ 0.2233, -0.1955, -0.3990,  ..., -0.1488, -0.3951, -0.1991],
        [ 0.5356, -0.1535, -0.2723,  ..., -0.1189, -0.0980, -0.3061],
        [-0.0269, -0.1622, -0.7113,  ...,  0.2489,  0.0599,  0.1411]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [135]:
(1024**2 + 512**2  + 256**2 + 128**2) * 10

13926400

In [31]:
x_batch.shape

torch.Size([4, 6, 512])

In [33]:
tt = torch.cat((x_batch,x_batch))

In [34]:
tt.shape

torch.Size([8, 6, 512])

In [243]:
from torch import nn
import torch.nn.functional as F

class ResidualConvBlock2D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1):
        super().__init__()
        #self.padding=padding
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=1, padding=0, dilation=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        
        if in_channels != out_channels or kernel_size!=1 or stride!=1 or padding!=0 or dilation!=1:
            self.downsample = nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding, dilation=dilation) 
        else:
            self.downsample = nn.Identity()

        '''
        self.downsample = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1),
                nn.BatchNorm2d(out_channels)
            )
        '''

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.downsample(residual)
        out = F.relu(out)
        return out

class MyEmbeddingConv2D(nn.Module):
    def __init__(self, nout, **kwargs):
        super().__init__()

        self.nout = nout
        self.layers = nn.ModuleList([self.make_layer()])
        self.linear = nn.Linear(3, self.nout)

    def make_layer(self):
        layers = []
        layers.append(ResidualConvBlock2D(in_channels=1, out_channels=8,
                                          kernel_size=(3,2), stride=(1,1), dilation=(1,1), padding = (2,0)))
        layers.append(ResidualConvBlock2D(in_channels=8, out_channels=16,
                                          kernel_size=(3,4), stride=(1,1), dilation=(2,1), padding = (2,0)))
        layers.append(ResidualConvBlock2D(in_channels=16, out_channels=16,
                                          kernel_size=(3,2), stride=(1,1), dilation=(2,1), padding = (2,0)))
        layers.append(ResidualConvBlock2D(in_channels=16, out_channels=3,
                                          kernel_size=(3,2), stride=(1,1), dilation=(1,1), padding = (2,0)))
        return nn.Sequential(*layers)

    def forward(self, x):
        # x : [batch_size, channel (det_123, amp/phase) = 2*ndet, length (number of samples)]
        # bs,_,_,_  = x.shape
        x = x.unsqueeze(1)
        for layer in self.layers:
            x = layer(x)
        x = F.avg_pool2d(x, x.size()[2:])
        x = x.view(x.size(0), -1)
        output = self.linear(x)
        return output

In [147]:
x_test = x_batch.unsqueeze(1)

In [148]:
x_test.shape

torch.Size([4, 1, 6, 512])

In [244]:
embd = MyEmbeddingConv2D(nout=256).to(device)


In [245]:
count_parameters(embd)

8726

In [227]:
torch.nn.Conv2d?

In [165]:
layer1 = torch.nn.Conv2d(1, 8, kernel_size=(3,2), stride=(1,1), dilation=(1,1), padding = (2,0)).to(device)

In [220]:
layer1 = ResidualConvBlock2D(1, 8, kernel_size=(3,2), stride=(4,2), dilation=(3,4), padding = (3,2)).to(device)

In [246]:
a = embd(x_batch)

In [247]:
a.shape

torch.Size([4, 256])

In [188]:
import glasflow

In [190]:
glasflow.nflows.nn.nets.ConvResidualNet??

In [146]:
nn.Conv2d?

In [138]:
condition = embd(x_batch.unsqueeze(1))

RuntimeError: The size of tensor a (255) must match the size of tensor b (512) at non-singleton dimension 3

In [110]:
condition.shape

torch.Size([4, 256])

In [133]:
F.avg_pool2d?

In [144]:
aa = torch.tensor([[[1+1j, 2, 3, 4, 5, 6, 7]]], dtype=torch.complex64)
F.avg_pool1d(aa, kernel_size=3, stride=2)

RuntimeError: "avg_pool2d" not implemented for 'ComplexFloat'